In [1]:
### ================
### 
### [5.0, 4.0, 1.0, 2.0, 3.0] - 打分列表
### 等于1分的 - label =0
### 等于5分的 - Label =2
### 等于3分的 - label =1
### 2019-11-25
### Liu,Yazhou

In [1]:
import numpy as np
import pandas as pd
import pandas as pd 
import numpy as np 
import jieba
import multiprocessing
import re
import time
import os
from gensim.models.word2vec import Word2Vec
from gensim.corpora.dictionary import Dictionary
from sklearn.cross_validation import train_test_split
from tensorflow.python.keras.models import Sequential,model_from_yaml
from gensim.models import KeyedVectors
from tensorflow.python.keras.layers import Dense, GRU, Embedding, LSTM, Bidirectional,Dropout,Activation
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.optimizers import RMSprop
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.utils import to_categorical
from tensorflow.python.keras.models import save_model
from tensorflow.python.keras.models import load_model

C:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### 1.载入原始数据

In [16]:
path = './main _chinese_sentiment_datas/yf_amazon/ratings/'
DataFrame = pd.read_csv(open(os.path.join(path, "ratings.csv"), 'r', 
                        encoding='utf-8'),
                        index_col=0)

### 1.1 去掉任何行或者列为空的情况

In [24]:
DataFrame = DataFrame.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)

In [18]:
DataFrame.head()

,productId,rating,timestamp,title,comment
userId,,,,,
15905.0,452609,5.0,1380988800,很喜欢,"很好很强大,纸张超赞不是一般画册所能比拟的,图片很好,特点基本都表现了出来。物种很全"
94522.0,452609,5.0,1333123200,精品!,"买过很多本DK的百科全书类图书,这本书是我买过的最经典的一本!很厚的一本书,装帧很漂亮!内容..."
317087.0,452609,5.0,1346688000,不错!!!!!!!!!!,"没想到送货的时候正好是下雨天,而且书没有塑料包装。 但是送货人员还是很小心的,虽然外面的纸箱..."
502593.0,452609,5.0,1347638400,很值的一本书,"是本很值得收藏的书。 本身DK的eyewitness系列就很值得收藏,这套书都秉承着科普、图..."
1098610.0,452609,5.0,1349280000,本书书评,"本书非常棒,经典就是不一样。"


### 1.2 将评分+title+comments 存入文件300W数据

In [11]:
LIMITS  = 1e4*300  #300W 数据 = 2966810
ratings  = DataFrame.rating.values.tolist()
titles   = DataFrame.title.values.tolist()
comments = DataFrame.comment.values.tolist() 
productIds = DataFrame.productId.values.tolist()
writor = open('./main _chinese_sentiment_datas/yf_amazon/ratings/ratings_titles_comments_3.txt','w',encoding='utf-8')
writmerge = open('./main _chinese_sentiment_datas/yf_amazon/ratings/ratings_titlesmergecomments_3.txt','w',encoding='utf-8')
for i in range(len(ratings)):
    if i<=LIMITS:
        writor.write(str(ratings[i])+'||'+titles[i]+'||'+comments[i]+'\n')
        writmerge.write(str(ratings[i])+'||'+titles[i]+','+comments[i]+'\n')#将title+comments merge一起

writor.close()
writmerge.close()

### 1.3 在后100W数据中记录下来

In [26]:
LIMITS  = 1e4*300  #300W 数据 = 2966810
ratings  = DataFrame.rating.values.tolist()
titles   = DataFrame.title.values.tolist()
comments = DataFrame.comment.values.tolist() 
productIds = DataFrame.productId.values.tolist()

writmerge = open('./main _chinese_sentiment_datas/yf_amazon/ratings/ratings_additional100W_3.txt','w',encoding='utf-8')
for i in range(len(ratings)):
    if i>LIMITS:
        writmerge.write(str(ratings[i])+'||'+titles[i]+','+comments[i]+'\n')#将title+comments merge一起

writmerge.close()

In [8]:
m,j,k

(43430, 123519, 607370)

In [27]:
43430+123519+607370,len(ratings),j

(774319, 4081136, 43430)

### 2.数据清洗后获得文本tokens

In [2]:
def get_cn_model(): 
    cn_model = KeyedVectors.load_word2vec_format('embeddings/sgns.zhihu.bigram', 
                                             binary=False, unicode_errors="ignore")
    
    return cn_model

def re_sub(inputs):
    ## inputs is one list and outputs is also
    texts  = inputs
    texts = re.sub("\//@[a-zA-Z\W+]+", "",texts)
    texts = re.sub("\@[a-zA-Z\w+]+", "",texts)
    texts = re.sub("[\-\#+\//@.\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？、""【】~@#￥%……&*（）]+","",texts)
    texts = re.sub('[A-Za-z0-9\!\%\[\]\,\。\;\:\::\?\“\”\”“\～+\:?\;;\>>]','',texts)
    texts = re.sub('\：：?','',texts)
    texts = re.sub('\·?\－\<\·\\‘\’\?\····.?\···\×\=?','',texts)
    texts = re.sub('\－.?','',texts)
    texts = re.sub('[^0-9A-Za-z\u4e00-\u9fa5]','',texts)# 留下中文，数字，大小写
    return texts

def get_train_data(path,texts_clean,texts_target,NUMS_FORONE_CATEGORY,LENS = 2):
    i = 0
    j = 0
    k = 0
    with open(path,'r',encoding = 'utf-8') as file:
        for line in file.readlines():
            items = line.split('||')
            text = items[1]
            text = re_sub(text)
            if len(text)>LENS and text!='\n':
                if eval(items[0])==1:
                    i+=1
                    if i<=NUMS_FORONE_CATEGORY:
                        texts_target.append(0)
                        texts_clean.append(text)
                elif eval(items[0])==3:
                    j+=1
                    if j<=NUMS_FORONE_CATEGORY:
                        texts_target.append(1)  #texts_target 出现数字
                        texts_clean.append(text)
                elif eval(items[0])==5:
                    k+=1
                    if k<=NUMS_FORONE_CATEGORY:
                        texts_target.append(2)
                        texts_clean.append(text)
                else:
                    pass
                
    return texts_target,texts_clean

def get_train_tokens(train_tokens,texts_clean,cn_model,LENS =1):
    for text in texts_clean:
        cut = jieba.cut(text)
        cut_list = [i for i in cut]
        for i, word in enumerate(cut_list):
            try:
                cut_list[i] = cn_model.vocab[word].index
            except KeyError:
                cut_list[i] = 0
        train_tokens.append(cut_list)
    return train_tokens

In [3]:
cn_model = get_cn_model()

### 2.1样本乱序处理 - 只运行一次

In [4]:
### 对原来的数据样本乱序 ### 
import random
def shuffles(inputs,outputs):
    contents = []
    file1 = open(inputs, 'r', encoding='utf-8') 
    file2 = open(outputs, 'w', encoding='utf-8')
    for line in file1.readlines():
        contents.append(line)
    
    random.shuffle(contents)
    for content in contents:
        file2.write(content)
    
    file1.close()
    file2.close()

# 样本乱序后存放在
raw_data = './main _chinese_sentiment_datas/yf_amazon/ratings/ratings_titlesmergecomments_3.txt'
random_data = './main _chinese_sentiment_datas/yf_amazon/ratings/ratings_titlesmergecomments_3_random.txt'
#shuffles(raw_data,random_data)

### 2.2 获取Tokens 数据

In [4]:
random_data = './main _chinese_sentiment_datas/yf_amazon/ratings/ratings_titlesmergecomments_3.txt'
texts_clean = []
texts_target= []
train_tokens = []
NUMS_FORONE_CATEGORY = 1e4*12 #每个分类的数量控制住
texts_target,texts_clean = get_train_data(random_data,texts_clean,texts_target,NUMS_FORONE_CATEGORY)

In [5]:
train_tokens = get_train_tokens(train_tokens,texts_clean,cn_model)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\H155809\AppData\Local\Temp\jieba.cache
Loading model cost 0.705 seconds.
Prefix dict has been built succesfully.


In [6]:
len(texts_clean),len(texts_target),len(train_tokens)

(360000, 360000, 360000)

### 2.3 将清洗后的文本及评分标签记录下

In [61]:
path = './main _chinese_sentiment_datas/yf_amazon/ratings/ratings_titlesmergecomments_3_clean.txt'
writes = open(path,'w',encoding='utf-8')
i= -1
for line in texts_clean:
    i+=1
    writes.write(str(texts_target[i])+'||'+line+'\n')
writes.close()

### 3.计算词汇表大小和词长度

In [7]:
def max_token(train_tokens):
    num_tokens = [ len(tokens) for tokens in train_tokens ]
    num_tokens = np.array(num_tokens)
    max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
    max_tokens = int(max_tokens)
    return max_tokens

def vocab_size(train_tokens):
    vocab_tokens = []
    i = 0
    t1 = time.time()
    for tokens in train_tokens:
        for token in tokens:
            i+=1
            if token not in vocab_tokens:
                vocab_tokens.append(token)
            else:
                pass
        if i%(1e5) == 0:
            print("处理前%d的词花费的时间是:%0.2f"%(i,(time.time() - t1)/60),'mins')
    print("词汇表大小是：",len(vocab_tokens))
    return len(vocab_tokens)

In [10]:
max_tokens = max_token(train_tokens)
max_tokens

111

In [11]:
num_words = vocab_size(train_tokens)
num_words

处理前400000的词花费的时间是:0.24 mins
处理前3900000的词花费的时间是:4.65 mins
处理前6200000的词花费的时间是:7.96 mins
处理前6900000的词花费的时间是:9.03 mins
处理前7600000的词花费的时间是:10.08 mins
处理前7800000的词花费的时间是:10.38 mins
处理前9000000的词花费的时间是:12.42 mins
词汇表大小是： 85427


85427

### 4 引入词嵌入矩阵获取特征并由样本Padding 

In [4]:
def get_embedding_matrix(embedding_dim,num_words,cn_model):
    embedding_matrix = np.zeros((num_words, embedding_dim))
    for i in range(num_words):
        embedding_matrix[i,:] = cn_model[cn_model.index2word[i]]
    embedding_matrix = embedding_matrix.astype('float32')
    return embedding_matrix

def add_padding(train_tokens,train_target,max_tokens,num_words):
    train_pad = pad_sequences(train_tokens, maxlen=max_tokens,
                            padding='pre', truncating='pre')
    train_pad[train_pad>=num_words] = 0
    train_targets = np.array(train_target)
    return train_pad,train_targets
def model(y_train,y_test,X_train,X_test,epochs,batch_size,num_classes):#mask_zero = True
    model = Sequential()
    model.add(Embedding(num_words,
                        embedding_dim,
                        mask_zero= True,
                        weights=[embedding_matrix],
                        input_length=max_tokens,
                        trainable=False))##,
    model.add(Bidirectional(LSTM(units= 64, return_sequences=True)))
    model.add(LSTM(units=16, return_sequences=False))
    model.add(Dense(3, activation='softmax')) # Dense全连接层，输出维度是3
    #model.add(Activation('softmax'))
    optimizer = Adam(lr=1*1e-2)
    model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])
    
    model.summary()
    path_checkpoint = './main_model/sentiment_checkpoint_Amazon3cates.keras'
    checkpoint = ModelCheckpoint(filepath=path_checkpoint, monitor='val_loss',
                                      verbose=1, save_weights_only=True,
                                      save_best_only=True)
    try:
        model.load_weights(path_checkpoint)
    except Exception as e:
        print(e)
        
    earlystopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
    lr_reduction  = ReduceLROnPlateau(monitor='val_loss',
                                       factor=0.1, min_lr=1e-8, patience=0,
                                       verbose=1)
    callbacks = [earlystopping, checkpoint,lr_reduction]
    y_train_ = to_categorical(y_train,num_classes = num_classes)
    y_test_  = to_categorical(y_test,num_classes = num_classes)
    
    model.fit(X_train, y_train_,validation_split=0.2, epochs=epochs,
          batch_size=batch_size,
          callbacks=callbacks)
    
    save_model(model,'./main_model/sentiment_checkpoint_Amazon3cates.h5')
    result = model.evaluate(X_test, y_test_)
    print('\nAccuracy is :{0:.2%}'.format(result[1]))
    return result[1]*100

### 5 训练前的数据准备

In [10]:
embedding_dim = 300
num_words     = 85427
max_tokens    = 111

embedding_matrix = get_embedding_matrix(embedding_dim,num_words,cn_model)
train_pad,texts_target_pad = add_padding(train_tokens,texts_target,max_tokens,num_words)
X_train, X_test, y_train, y_test = train_test_split(train_pad, # array
                                                    texts_target,  # array
                                                    test_size=0.2,
                                                    random_state=300)

In [10]:
len(train_tokens),len(texts_target),len(train_pad)

(360000, 360000, 360000)

### 5.模型开始训练

In [12]:
epochs      = 2
batch_size  = 1024
num_classes = 3
#accuracy = model(y_train,y_test,X_train,X_test,epochs,batch_size,num_classes)

### 6.重新加载模型 - 继续训练

In [13]:
model_path = './main_model/sentiment_checkpoint_Amazon3cates.h5'
model = load_model(model_path)

In [14]:
batch_size  = 1024
num_classes = 3 
model_path = './main_model/sentiment_checkpoint_Amazon3cates.h5'


for i in range(1,5):
    if i%2 ==0:
        time.sleep(60*2)
    else:
        print('\n=========epochs========== ===>',i)
        model = load_model(model_path)
        X_train, X_test, y_train, y_test = train_test_split(train_pad, # array
                                                    texts_target,  # array
                                                    test_size=0.2,
                                                    random_state=50*i + i)
        y_train_ = to_categorical(y_train,num_classes = num_classes)
        y_test_  = to_categorical(y_test,num_classes = num_classes)
        model.fit(X_train, y_train_,validation_split=0.2, epochs=i,
          batch_size=batch_size)
        save_model(model,'./main_model/sentiment_checkpoint_Amazon3cates.h5')
        result = model.evaluate(X_test, y_test_)
        print('\nAccuracy is :{0:.2%}'.format(result[1]))


=========epochs========== ===> 1
Train on 230400 samples, validate on 57600 samples
Epoch 1/1
72000/72000 [==============================] - 183s   

Accuracy is :86.08%

=========epochs========== ===> 3
Train on 230400 samples, validate on 57600 samples
Epoch 1/3
230400/230400 [==============================] - 1698s - loss: 0.3739 - acc: 0.8621 - val_loss: 0.3698 - val_acc: 0.8628
Epoch 2/3
230400/230400 [==============================] - 1728s - loss: 0.3737 - acc: 0.8623 - val_loss: 0.3698 - val_acc: 0.8628
Epoch 3/3
72000/72000 [==============================] - 185s   

Accuracy is :85.90%


In [12]:
save_model(model,'./main_model/sentiment_checkpoint_Amazon3cates.h5')

In [14]:
result = model.evaluate(X_test, y_test_)
print('Accuracy is :{0:.2%}'.format(result[1]))

72000/72000 [==============================] - 199s   

Accuracy is :85.90%


### 6 在个别新的样例上测试结果 =>效果不错

In [15]:
def get_cn_model(): 
    cn_model = KeyedVectors.load_word2vec_format('embeddings/sgns.zhihu.bigram', 
                                             binary=False, unicode_errors="ignore")
    
    return cn_model

def re_sub(inputs):
    ## inputs is one list and outputs is also
    texts  = inputs
    texts = re.sub("\//@[a-zA-Z\W+]+", "",texts)
    texts = re.sub("\@[a-zA-Z\w+]+", "",texts)
    texts = re.sub("[\-\#+\//@.\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？、""【】~@#￥%……&*（）]+","",texts)
    texts = re.sub('[A-Za-z0-9\!\%\[\]\,\。\;\:\::\?\“\”\”“\～+\:?\;;\>>]','',texts)
    texts = re.sub('\：：?','',texts)
    texts = re.sub('\·?\－\<\·\\‘\’\?\····.?\···\×\=?','',texts)
    texts = re.sub('\－.?','',texts)
    texts = re.sub('[^0-9A-Za-z\u4e00-\u9fa5]','',texts)# 留下中文，数字，大小写
    return texts

def prediction(text,LENS =2):
    cn_model             = get_cn_model()
    text = re_sub(text)
    if len(text)>LENS and text!='\n' and text!='\t':
        cut_list = [i for i in jieba.cut(text)]
        for i, word in enumerate(cut_list):
            try:
                cut_list[i] = cn_model.vocab[word].index
                if cut_list[i]>=85427:  #nums word
                    cut_list[i] = 0
            except KeyError:
                cut_list[i] = 0
            
    #padding
    max_tokens = 111  #get from above model parameters
    train_pad = pad_sequences([cut_list], maxlen=max_tokens,
                            padding='pre', truncating='pre')
    
    # loading model
    from tensorflow.python.keras.models import load_model
    model_path = './main_model/sentiment_checkpoint_Amazon3cates.h5'
    model = load_model(model_path)
    
    #result = model.predict(x=train_pad)
    label = model.predict_classes(x=train_pad)
    
    #coefs = result[0][0]
    return label[0]  # 返回判断的阈值

In [16]:
text_list = [
    # 1
    '颜色不好看,送给我的婆婆和妈妈了。不适合我',
    '退都不能退!!!,根本不是所谓的淡紫色,明明就是咖色',
    '很垃圾的产品,根本就没有大家评论的那么好 怀疑她们都是托 图到嘴唇上很粘稠 根本就涂不开 什么光泽度也没有 大家不要听那些好评的 我花了44买的真亏 打水漂了',
    '颜色不好,不好用,颜色不好看,直接丢了',
    # 3
    '差强人意,颜色不好看,开始我还以为发错颜色了,结果一对比膏体上的货号名称,确认就是樱桃红,一点都不像樱桃那种红色,倒有点像玫瑰红,暗沉,不像樱桃那么娇艳,不喜欢,但是包装和送货还是不错的,总之差强人意吧!',
    '颜色不是特别喜欢,买回来后,觉得跟图示的颜色有出入,颜色不是特别喜欢,一直没用',
    '不是很好,不是特别喜欢这个色,选 错了号了',
    '露华浓,不是唇蜜。。不是我想象中的效果。。。不是很好。',
    # 5
    '颜色还可以,颜色还可以就是有点深,质感挺好的。',
    '喜欢,女儿说挺好用的,有点闪亮闪亮的,颜色也挺适合!',
    '颜色很正,感觉比较好用,很方便',
    '非常喜欢啊!很美呢?,和板友去玩的时候涂的这个,都说好看,我个人是非常喜欢的,玩滑板都特别起劲。哈哈!颜色超级无敌美。'
]

results = []
for text in text_list:
    label = prediction(text)
    results.append(label) 

1/1 [==============================] - 1s


In [17]:
cte = {2:'正面评价',1:'中性评价',0:'负面评价'}
i = 0
for label in results:
    i+=1
    print("第%d条评价是:%s"%(i,text_list[i-1]) )
    print("模型判断这是一条: ##%s##"%(cte[label]))
    print('\n')

第1条评价是:颜色不好看,送给我的婆婆和妈妈了。不适合我
模型判断这是一条: ##负面评价##


第2条评价是:退都不能退!!!,根本不是所谓的淡紫色,明明就是咖色
模型判断这是一条: ##负面评价##


第3条评价是:很垃圾的产品,根本就没有大家评论的那么好 怀疑她们都是托 图到嘴唇上很粘稠 根本就涂不开 什么光泽度也没有 大家不要听那些好评的 我花了44买的真亏 打水漂了
模型判断这是一条: ##负面评价##


第4条评价是:颜色不好,不好用,颜色不好看,直接丢了
模型判断这是一条: ##负面评价##


第5条评价是:差强人意,颜色不好看,开始我还以为发错颜色了,结果一对比膏体上的货号名称,确认就是樱桃红,一点都不像樱桃那种红色,倒有点像玫瑰红,暗沉,不像樱桃那么娇艳,不喜欢,但是包装和送货还是不错的,总之差强人意吧!
模型判断这是一条: ##中性评价##


第6条评价是:颜色不是特别喜欢,买回来后,觉得跟图示的颜色有出入,颜色不是特别喜欢,一直没用
模型判断这是一条: ##中性评价##


第7条评价是:不是很好,不是特别喜欢这个色,选 错了号了
模型判断这是一条: ##中性评价##


第8条评价是:露华浓,不是唇蜜。。不是我想象中的效果。。。不是很好。
模型判断这是一条: ##中性评价##


第9条评价是:颜色还可以,颜色还可以就是有点深,质感挺好的。
模型判断这是一条: ##中性评价##


第10条评价是:喜欢,女儿说挺好用的,有点闪亮闪亮的,颜色也挺适合!
模型判断这是一条: ##正面评价##


第11条评价是:颜色很正,感觉比较好用,很方便
模型判断这是一条: ##正面评价##


第12条评价是:非常喜欢啊!很美呢?,和板友去玩的时候涂的这个,都说好看,我个人是非常喜欢的,玩滑板都特别起劲。哈哈!颜色超级无敌美。
模型判断这是一条: ##正面评价##




### 取一些额外的数据[13W+]继续训练 

In [18]:
path_new = './main _chinese_sentiment_datas/yf_amazon/ratings/ratings_additional12W_3.txt'

texts_clean = []
texts_target= []
train_tokens = []
texts_target,texts_clean = get_train_data(path_new,texts_clean,texts_target,43148,LENS = 2)

In [19]:
train_tokens = get_train_tokens(train_tokens,texts_clean,cn_model)

In [20]:
len(texts_clean),len(texts_target),len(train_tokens)

(129444, 129444, 129444)

In [9]:
embedding_dim = 300
num_words     = 85427
max_tokens    = 111
embedding_matrix = get_embedding_matrix(embedding_dim,num_words,cn_model)
train_pad,texts_target_pad = add_padding(train_tokens,texts_target,max_tokens,num_words)

In [12]:
batch_size  = 1024
num_classes = 3 
model_path = './main_model/sentiment_checkpoint_Amazon3cates.h5'


for i in range(1,3):
    if i%2 ==0:
        time.sleep(60*2)
    else:
        print('\n========= epochs ===============',i)
        model = load_model(model_path)
        X_train, X_test, y_train, y_test = train_test_split(train_pad, # array
                                                    texts_target,  # array
                                                    test_size=0.2,
                                                    random_state=560*i + i)
        y_train_ = to_categorical(y_train,num_classes = num_classes)
        y_test_  = to_categorical(y_test,num_classes = num_classes)
        model.fit(X_train, y_train_,validation_split=0.1, epochs=i,
          batch_size=batch_size)
        save_model(model,'./main_model/sentiment_checkpoint_Amazon3cates.h5')
        result = model.evaluate(X_test, y_test_)
        print('Accuracy is :{0:.2%}'.format(result[1]))


========= epochs =============== 1
Train on 93751 samples, validate on 10417 samples
Epoch 1/1
26043/26043 [==============================] - 69s    
Accuracy is :81.91%
